In [202]:
import pandas as pd
import time

In [203]:
old=pd.read_json('data/old.json')#.reset_index(drop=True)
old.number_of_citations.sum()

1638

Download citations of each paper

In [204]:
UPDATE=False
if UPDATE:
    for r in old['recid']:
        p=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r)
                      ).to_json('data/{}.json'.format(r))
        print(r)
        time.sleep(1)

Download INSPIRE HEP Profile

In [205]:
# TODO: Check if the author has more that 250 records!
exact_author='D.Restrepo.1'
url='http://inspirehep.net/search?p=ea+{}&rg=250&of=recjson'.format(exact_author)
df=pd.read_json(url)
new=df.copy()
df.number_of_citations.sum()

1642

author='restrepo,+d'
df=pd.read_json(
    "http://inspirehep.net/search?ln=en&ln=en&p=a+{}+AND+collection%3Aciteable&rg=250&of=recjson".format(
        author))
new=df.copy()

In [206]:
import sys
ANALYSIS=False
if new['number_of_citations'].sum()>old['number_of_citations'].sum():
    ANALYSIS=True
if not ANALYSIS:
    sys.exit('NO NEW CITES TO ANALYSE!')

In [207]:
old['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]
new['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]

In [208]:
old=old.rename({'number_of_citations':'nc'},axis='columns') ##add other relevant columns for analysis

In [209]:
ANALYSIS

True

In [210]:
new=new.merge(old[['nc','iid']],on='iid',how='left')#[['number_of_citations','nc']]

In [211]:
news=new[new['number_of_citations']>new['nc']]

In [212]:
print('New citations in:')
news['recid']
#['primary_report_number'].apply(lambda l:  [x for x in l if x.find('arXiv')>-1])

New citations in:


0    1755250
2    1741056
3    1708907
8    1663960
Name: recid, dtype: int64

In [216]:
for r in news['recid']:
    #Analysis of each citation
    print('===== NEW CITATION FOR {}====='.format(r))
    oldrefersto=pd.read_json('data/{}.json'.format(r))
    refersto=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r))
    print( news[news['recid']==r]['title'].values[0] )
    print('with')
    print(news[news['recid']==r]['authors'].apply(lambda l: [d['full_name'] for d in l]).values[0])
    print('WAS CITED BY:')
    try:
        newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'])
    except KeyError:
        newrecid=pd.np.setdiff1d(refersto['recid'],[])
    #newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'][:-2])
    for rr in newrecid:
        print('*'*60)
        print( refersto[refersto['recid']==rr]['title'].values[0])
        print( refersto[refersto['recid']==rr]['authors'].apply(lambda l: 
                                            [d['full_name'] for d in l]).values[0])
        print( refersto[refersto['recid']==rr]['primary_report_number'].apply(lambda l: 
                                            [x for x in l if x.find('arXiv')>-1]).values[0] )    
    print('UPDATE CITATIONS (Y)')
    YN=input('Y/N')
    if YN!='N' or YN!='n' or YN!='not' or YN!='Not':
        print('UPDATING JSON DATA CITATION for {}'.format(r))
        refersto.to_json('data/{}.json'.format(r))

===== NEW CITATION FOR 1755250=====
{'title': 'Dirac neutrino mass generation from Majorana messenger'}
with
['Calle, Julian', 'Restrepo, Diego', 'Zapata, Óscar']
WAS CITED BY:
************************************************************
{'title': 'Minimal Realizations of Dirac Neutrino Mass from Generic One-loop and Two-loop Topologies at $d=5$'}
['Jana, Sudip', 'K., Vishnu P.', 'Saad, Shaikh']
['arXiv:1910.09537']
UPDATE CITATIONS (Y)
Y/N
UPDATING JSON DATA CITATION for 1755250
===== NEW CITATION FOR 1741056=====
{'title': 'Singlet-Doublet Dirac Dark Matter and Neutrino Masses'}
with
['Restrepo, Diego', 'Rivera, Andrés', 'Tangarife, Walter']
WAS CITED BY:
************************************************************
{'title': 'Minimal Realizations of Dirac Neutrino Mass from Generic One-loop and Two-loop Topologies at $d=5$'}
['Jana, Sudip', 'K., Vishnu P.', 'Saad, Shaikh']
['arXiv:1910.09537']
UPDATE CITATIONS (Y)
Y/N
UPDATING JSON DATA CITATION for 1741056
===== NEW CITATION FOR 170

In [217]:
print('UPDATE FULL DATABASE for profiles (N)')
YN=input('Y/N')
if YN=='Y' or YN=='y' or YN=='yes' or YN=='Yes':
    df.to_json('data/old.json')

UPDATE FULL DATABASE for profiles (N)
Y/Ny
